# SPY Spreadsheet

In [1]:
# use future imports for python 3.x forward compatibility
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

# other imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import itable

# project imports
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [5]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [6]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [7]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,26.60,26.47,26.60,26.58,1003200.00,26.58
1993-02-01,26.77,26.60,26.60,26.77,480500.00,26.77
1993-02-02,26.85,26.70,26.75,26.83,201300.00,26.83
1993-02-03,27.13,26.85,26.87,27.11,529400.00,27.11
1993-02-04,27.28,26.91,27.21,27.23,531500.00,27.23


Add technical indicator: 200 day MA

In [8]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [9]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94,287.56
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35,287.80
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00,288.04
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10,288.27
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48,288.49


Add technical indicator: ATR

In [10]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [11]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr
date,,,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94,287.56,2.42
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35,287.80,2.36
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00,288.04,2.33
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10,288.27,2.29
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48,288.49,2.24


Add technical indicator: 5 day high, and 5 day low

In [12]:
high5 = pd.Series(ts.high).rolling(window=5).max()
low5 = pd.Series(ts.low).rolling(window=5).min()
ts['high5'] = high5
ts['low5'] = low5

In [13]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94,287.56,2.42,309.65,306.06
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35,287.80,2.36,309.65,306.06
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00,288.04,2.33,309.99,306.06
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10,288.27,2.29,309.99,307.03
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48,288.49,2.24,309.99,307.03


Add technical indicator: RSI, and 2-period cumulative RSI

In [14]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.Series(ts.rsi2).rolling(window=2).sum()
ts['c2rsi2'] = c2rsi2

In [15]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94,287.56,2.42,309.65,306.06,96.41,189.36
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35,287.80,2.36,309.65,306.06,54.74,151.15
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00,288.04,2.33,309.99,306.06,76.82,131.56
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10,288.27,2.29,309.99,307.03,79.84,156.66
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48,288.49,2.24,309.99,307.03,89.82,169.67


Add technical indicator: Midpoint

In [16]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [17]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94,287.56,2.42,309.65,306.06,96.41,189.36,308.01
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35,287.80,2.36,309.65,306.06,54.74,151.15,307.90
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00,288.04,2.33,309.99,306.06,76.82,131.56,309.07
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10,288.27,2.29,309.99,307.03,79.84,156.66,308.60
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48,288.49,2.24,309.99,307.03,89.82,169.67,308.87


Add technical indicator: SMA10 of midpoint

In [18]:
sma10mp = pd.Series(ts.mp).rolling(window=10).mean()
ts['sma10mp'] = sma10mp

In [19]:
ts.head(10)

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1993-01-29,26.60,26.47,26.60,26.58,1003200.00,26.58,nan,nan,nan,nan,nan,nan,26.54,nan
1993-02-01,26.77,26.60,26.60,26.77,480500.00,26.77,nan,nan,nan,nan,nan,nan,26.69,nan
1993-02-02,26.85,26.70,26.75,26.83,201300.00,26.83,nan,nan,nan,nan,100.00,nan,26.77,nan
1993-02-03,27.13,26.85,26.87,27.11,529400.00,27.11,nan,nan,nan,nan,100.00,200.00,26.99,nan
1993-02-04,27.28,26.91,27.21,27.23,531500.00,27.23,nan,nan,27.28,26.47,100.00,200.00,27.09,nan
1993-02-05,27.26,27.06,27.21,27.21,492100.00,27.21,nan,nan,27.28,26.60,89.34,189.34,27.16,nan
1993-02-08,27.30,27.17,27.21,27.21,596100.00,27.21,nan,nan,27.30,26.70,89.34,178.67,27.24,nan
1993-02-09,27.11,26.96,27.11,27.02,122100.00,27.02,nan,nan,27.30,26.85,16.96,106.30,27.04,nan
1993-02-10,27.08,26.94,27.02,27.06,379600.00,27.06,nan,nan,27.30,26.91,37.28,54.24,27.01,nan


Add technical indicator: Standard Deviation

In [20]:
sd = pd.Series(ts.mp).rolling(window=10).std()
ts['sd'] = sd

In [21]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94,287.56,2.42,309.65,306.06,96.41,189.36,308.01,305.68,2.23
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35,287.80,2.36,309.65,306.06,54.74,151.15,307.90,306.13,2.17
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00,288.04,2.33,309.99,306.06,76.82,131.56,309.07,306.68,2.14
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10,288.27,2.29,309.99,307.03,79.84,156.66,308.60,307.22,1.83
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48,288.49,2.24,309.99,307.03,89.82,169.67,308.87,307.81,1.11


In [22]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [23]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-11-08,309.00,307.03,307.80,308.94,49032100.00,308.94,287.56,2.42,309.65,306.06,96.41,189.36,308.01,305.68,2.23,310.13,301.23
2019-11-11,308.54,307.27,307.42,308.35,35797300.00,308.35,287.80,2.36,309.65,306.06,54.74,151.15,307.90,306.13,2.17,310.46,301.80
2019-11-12,309.99,308.15,308.75,309.00,46484600.00,309.00,288.04,2.33,309.99,306.06,76.82,131.56,309.07,306.68,2.14,310.96,302.41
2019-11-13,309.54,307.66,307.91,309.10,53917700.00,309.10,288.27,2.29,309.99,307.03,79.84,156.66,308.60,307.22,1.83,310.89,303.55
2019-11-14,309.64,308.09,308.79,309.48,44419328.00,309.48,288.49,2.24,309.99,307.03,89.82,169.67,308.87,307.81,1.11,310.03,305.59


Select a smaller time from for use with itable

In [24]:
df = ts['2019-01-01':]

In [25]:
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-01-02,247.75,242.57,242.60,246.74,126925200.00,246.74,267.77,6.11,247.94,230.54,84.93,168.27,245.16,243.21,5.18,253.57,232.85
2019-01-03,245.15,240.32,244.81,240.85,144140700.00,240.85,267.66,6.13,247.94,235.67,15.82,100.75,242.73,242.42,4.48,251.38,233.47
2019-01-04,249.63,243.77,244.18,248.92,142628800.00,248.92,267.58,6.32,249.63,240.32,73.94,89.76,246.70,242.15,4.07,250.29,234.01
2019-01-07,252.43,248.23,249.21,250.88,103139100.00,250.88,267.53,6.17,252.43,240.32,80.49,154.43,250.33,242.85,4.83,252.51,233.20
2019-01-08,253.77,250.51,253.29,253.24,102512600.00,253.24,267.51,5.96,253.77,240.32,87.84,168.33,252.14,243.92,5.60,255.13,232.71


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [26]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [27]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2019/01/02,247.75,242.57,242.60,246.74,126925200,246.74,267.77,6.11,247.94,230.54,84.93,168.27,245.16,243.21,5.18,253.57,232.85
2019/01/03,245.15,240.32,244.81,240.85,144140700,240.85,267.66,6.13,247.94,235.67,15.82,100.75,242.73,242.42,4.48,251.38,233.47
2019/01/04,249.63,243.77,244.18,248.92,142628800,248.92,267.58,6.32,249.63,240.32,73.94,89.76,246.70,242.15,4.07,250.29,234.01
2019/01/07,252.43,248.23,249.21,250.88,103139100,250.88,267.53,6.17,252.43,240.32,80.49,154.43,250.33,242.85,4.83,252.51,233.20
2019/01/08,253.77,250.51,253.29,253.24,102512600,253.24,267.51,5.96,253.77,240.32,87.84,168.33,252.14,243.92,5.60,255.13,232.71
2019/01/09,255.35,252.67,254.02,254.42,95006600,254.42,267.53,5.73,255.35,240.32,91.17,179.01,254.01,245.89,5.30,256.49,235.29
2019/01/10,255.59,251.98,252.73,255.32,96823900,255.32,267.52,5.58,255.59,243.77,93.77,184.94,253.79,247.60,4.72,257.05,238.16
2019/01/11,255.45,253.49,254.13,255.42,73858100,255.42,267.53,5.32,255.59,248.23,94.15,187.91,254.47,249.02,4.40,257.82,240.23
2019/01/14,254.75,252.88,253.33,253.86,70908200,253.86,267.53,5.12,255.59,250.51,32.27,126.42,253.81,249.85,4.44,258.74,240.97
2019/01/15,257.11,254.26,254.27,256.77,85208300,256.77,267.54,4.99,257.11,251.98,80.39,112.67,255.69,250.88,4.49,259.86,241.91
